In [23]:
import pandas as pd
import altair as alt
import numpy as np

In [15]:
import eco_style
alt.themes.enable("dark")

ThemeRegistry.enable('dark')

In [52]:
df = pd.read_csv("excess-deaths-cumulative-economist-single-entity.csv")

df = df[df["Entity"] == "World"]
df.columns

['Entity', 'Code', 'Day', 'Cumulative excess deaths (central estimate)',
       'Total confirmed deaths due to COVID-19',
       'Cumulative excess deaths (95% CI, upper bound)',
       'Cumulative excess deaths (95% CI, lower bound)']

df = df.melt(id_vars=["Day"], value_vars=["Cumulative excess deaths (central estimate)", "Total confirmed deaths due to COVID-19", "Cumulative excess deaths (95% CI, upper bound)", "Cumulative excess deaths (95% CI, lower bound)"])
df['variable'] = df.variable.map({
    "Cumulative excess deaths (central estimate)": "Excess mortality (central estimate)",
    "Total confirmed deaths due to COVID-19": "Confirmed deaths",
    "Cumulative excess deaths (95% CI, upper bound)": "Excess mortality (upper bound)",
    "Cumulative excess deaths (95% CI, lower bound)": "Excess mortality (lower bound)"
})

df = df.dropna()
df.columns = ['date', 'series', 'value']

# add a label to the last date of every series
df['label'] = df.groupby("series")["value"].transform("last")
df['label'] = np.where(df['value'] == df['label'], df['series'], '')

base = alt.Chart(df).encode(
    x=alt.X("date:T", title="", axis=alt.Axis(format="%b %Y", labelSeparation=10)),
    y=alt.Y("value:Q", title="", axis=alt.Axis(format="~s")),
    color=alt.Color("series:N", title="", legend=None, scale=alt.Scale(range=["#36B7B4", "#F4C245", "#ffe9b1", "#ffe9b1"])),
)

line = base.mark_line().encode(
    strokeDash=alt.StrokeDash("series:N", legend=None, scale=alt.Scale(range=[[0], [0], [4], [4]])),
    size=alt.Size("series:N", legend=None, scale=alt.Scale(range=[2, 2, 0.5, 0.5])),
)

labels = base.mark_text(align="left", dx=5, dy=0).encode(
    text="label",
)

chart = line + labels

chart = chart.properties(
    width=750,
    height=350,
    background="rgba(0,0,0,0)"
)

chart.save("covid_excess_deaths.png", scale_factor=2.0)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
WARN Channel size should not be used with an unsorted discrete field.


In [22]:
df.groupby("series")["value"].transform("last")

0       27314822.0
2       27314822.0
9       27314822.0
16      27314822.0
23      27314822.0
           ...    
6812    19248276.0
6819    19248276.0
6826    19248276.0
6833    19248276.0
6840    19248276.0
Name: value, Length: 2439, dtype: float64

In [19]:
labels

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)